In [ ]:
import pandas as pd
import seaborn as sns
from scipy.stats import zscore
import numpy as np
#RF
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import RandomizedSearchCV
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix, accuracy_score
#DT
from sklearn.tree import DecisionTreeClassifier
from sklearn import tree
from sklearn import metrics
import matplotlib.pyplot as plt
import matplotlib.image as pltimg
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score
#KNN
from sklearn.neighbors import KNeighborsRegressor
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV
#Gen alg
import random
import warnings
warnings.filterwarnings('ignore')



In [ ]:
df = pd.read_csv ('CollegeDistance.csv', index_col=0)

### Pierwszy moduł - usuwanie braków danych


#### Usuwanie

In [ ]:
def drop_NAs(df):
    df=df.dropna()
    return df

#### Forward fill

In [ ]:
def forward_fill(df):
    df=df.fillna(method='ffill')
    return df

#### Backward_fill

In [ ]:
def backward_fill(df):
    df=df.fillna(method='bfill')
    return df    

### Drugi moduł - usuwanie outlierów

#### Brak

In [ ]:
def do_nothing(df):
    return df

#### Zasada trzech sigm

In [ ]:
def three_sigmas(df):
    z_scores = zscore(df[['score','unemp','wage','distance','tuition']])
    abs_z_scores = np.abs(z_scores)
    filtered_entries = (abs_z_scores < 3).all(axis=1)
    new_df = df[filtered_entries]
    return new_df

#### Eliminacja za pomocą N-tego kwantyli

In [ ]:
def n_quantile(df, kwantyl):
    for column in ['score','unemp','wage','distance','tuition']:
        filtered_entries = df[column].between(df[column].quantile(kwantyl), df[column].quantile(1-kwantyl))
        new_df = df[filtered_entries]
    return new_df    



Dalszy kod przekształcający dane

In [ ]:
def prepare_data(new_df):
    #usuwam zmienne tekstowe
    #ethnicity
    new_df["IsAfam"]=np.where(new_df.ethnicity=="afam", 1,0) 
    new_df["IsHispanic"]=np.where(new_df.ethnicity=="hispanic  ", 1,0) 
    new_df=new_df.drop(columns="ethnicity")
    #gender
    new_df["IsFemale"]=np.where(new_df.gender=="female", 1,0) 
    new_df=new_df.drop(columns="gender")
    new_df["Fcollege"]=np.where(new_df.fcollege=="yes", 1,0) 
    new_df=new_df.drop(columns="fcollege")
    new_df["Mcollege"]=np.where(new_df.mcollege=="yes", 1,0) 
    new_df=new_df.drop(columns="mcollege")
    new_df["Home"]=np.where(new_df.home=="yes", 1,0) 
    new_df=new_df.drop(columns="home")
    new_df["Urban"]=np.where(new_df.urban=="yes", 1,0) 
    new_df=new_df.drop(columns="urban")
    new_df["Income"]=np.where(new_df.income=="high", 1,0) 
    new_df=new_df.drop(columns="income")
    new_df["IsWest"]=np.where(new_df.region=="west", 1,0) 
    new_df=new_df.drop(columns="region")
    return new_df




### Funkcja dopasowania - accuracy

In [ ]:
def print_accuracy(y_test,y_pred):
    return metrics.accuracy_score(y_test, y_pred)

### Trzeci moduł - model uczenia maszynowego

#### KNN

In [ ]:
def calculate_knn_errors():
    error = []

    # Calculating error for K values between 1 and 40
    for i in range(1, 40):
        knn = KNeighborsClassifier(n_neighbors=i)
        knn.fit(X_train, y_train)
        pred_i = knn.predict(X_test)
        error.append(np.mean(pred_i != y_test))
    
    return error

In [ ]:
def choose_best_k(err):
    best_k = 0
    mean_err = 1

    for i in range(0, len(err)):
        if(err[i]<mean_err):
            mean_err = err[i]
            best_k = i + 1
            
    return best_k

In [ ]:
def randomize_k():
    randomized_k = random.randint(1,40)
    return randomized_k 

In [ ]:
def knn(k,X_train, X_test, y_train, y_test):
    knn1 = KNeighborsClassifier(n_neighbors=k)
    knn_fit = knn1.fit(X_train, y_train)
    return knn_fit

In [ ]:
def calculate_y_pred(model,X_train, X_test, y_train, y_test):   
    pred_i1 = model.predict(X_test)   
    return pred_i1

#### Random Forest

In [ ]:
def create_random_grid():
    # Liczba drzew
    n_estimators = [20,50,100,200,500]
    # Liczba cech brana za każdym razem pod uwagę
    max_features = [2,3,4,5]
    # Maksymalna liczba liści
    max_leaf_nodes = [2, 5, 10,15]
    # Minimalna wielkość liścia
    min_samples_leaf = [1, 2, 4,8]

    random_grid = {'n_estimators': n_estimators,
                   'max_features': max_features,
                   'max_leaf_nodes': max_leaf_nodes,
                   'min_samples_leaf': min_samples_leaf}
    
    return random_grid

In [ ]:
def randomized_grid(): #Forma mutacji - losowanie siatki do random forest
    # Liczba drzew
    n_estimators = [random.randint(10,200)]
    # Liczba cech brana za każdym razem pod uwagę
    max_features = [random.randint(2,7)]
    # Maksymalna liczba liści
    max_leaf_nodes = [random.randint(2,30)]
    # Minimalna wielkość liścia
    min_samples_leaf = [random.randint(1,10)]

    random_grid = {'n_estimators': n_estimators,
                   'max_features': max_features,
                   'max_leaf_nodes': max_leaf_nodes,
                   'min_samples_leaf': min_samples_leaf}
    
    return random_grid


In [ ]:
def rf_fit(rand_grid,X_train, X_test, y_train, y_test):
    rf = RandomForestClassifier(random_state = 42)
    randomForest = RandomizedSearchCV(estimator = rf, param_distributions = rand_grid, n_iter = 5, cv = 3, verbose=0, random_state=1)
    randomForest_fit = randomForest.fit(X_train, y_train)
    
    return randomForest_fit

In [ ]:
def choose_best_estim(rf_fit_model):
    bestRF = rf_fit_model.best_estimator_
    
    return bestRF

#### Decision tree

In [ ]:
def decicion_tree(X_train, X_test, y_train, y_test):
    dtree = DecisionTreeClassifier()#criterion="entropy", max_depth=3)
    dtree_fit = dtree.fit(X_train, y_train)
    
    return dtree_fit

In [ ]:
def randomized_decicion_tree(X_train, X_test, y_train, y_test):
    dtree = DecisionTreeClassifier(criterion="entropy",splitter="random")
    dtree_fit = dtree.fit(X_train, y_train)
    
    return dtree_fit

#### SVM

In [22]:
def svm(X_train, X_test, y_train, y_test):
    sc = StandardScaler()
    X_train_SVC = sc.fit_transform(X_train)
    X_test_SVC = sc.transform(X_test)
    classifier = SVC(kernel = 'linear', gamma="auto")
    svm_fit1 = classifier.fit(X_train, y_train)
    
    return svm_fit1

In [23]:
def randomized_svm(X_train, X_test, y_train, y_test):
    sc = StandardScaler()
    X_train_SVC = sc.fit_transform(X_train)
    X_test_SVC = sc.transform(X_test)
    classifier = SVC(kernel = 'rbf', gamma="auto", max_iter=random.randint(1,10))
    svm_fit1 = classifier.fit(X_train, y_train)
    
    return svm_fit1

## Część genetyczna

### Kodowanie cech

#### Etap I

In [24]:
def usun_na(genotype,df):
    if genotype[0]==0:
        df=drop_NAs(df)
        return df
    elif genotype[0]==1:
        df=forward_fill(df)
        return df
    else:
        df=backward_fill(df)
        return df   

#### Etap II

In [25]:
def usun_outliery(genotype,df):
    if genotype[1]==0:
        new_df=do_nothing(df)
        return new_df
    elif genotype[1]==1:
        new_df=three_sigmas(df)
        return new_df
    else:
        new_df=n_quantile(df,0.1)
        return new_df  

#### Etap III

In [26]:
def uzyskaj_wynik(genotype,X_train, X_test, y_train, y_test):
    if genotype[2]==0:
        err_knn = calculate_knn_errors()
        if genotype[3]==False:
            knn_classifier = knn(choose_best_k(err_knn),X_train, X_test, y_train, y_test)
        else:
            knn_classifier = knn(randomize_k(),X_train, X_test, y_train, y_test)

        pred_y_knn = calculate_y_pred(knn_classifier,X_train, X_test, y_train, y_test)
        return pred_y_knn
    elif genotype[2]==1:
        if genotype[3]==False:
            svm_fit = svm(X_train, X_test, y_train, y_test)
        else:
            svm_fit = randomized_svm(X_train, X_test, y_train, y_test)    
        y_pred_svm = calculate_y_pred(svm_fit,X_train, X_test, y_train, y_test)
        return y_pred_svm
    elif genotype[2]==2:
        if genotype[3]==False:
            est_rf = choose_best_estim(rf_fit(create_random_grid(),X_train, X_test, y_train, y_test))
        else:
            est_rf = choose_best_estim(rf_fit(randomized_grid(),X_train, X_test, y_train, y_test))    
        y_pred_rf = calculate_y_pred(est_rf,X_train, X_test, y_train, y_test)
        return y_pred_rf
    else:
        if genotype[3]==False:
            dt_fit = decicion_tree(X_train, X_test, y_train, y_test)
        else:
            dt_fit = randomized_decicion_tree(X_train, X_test, y_train, y_test)    
        y_pred_dt = calculate_y_pred(dt_fit,X_train, X_test, y_train, y_test)
        return y_pred_dt
    
        

#### Funkcja dopasowania

In [27]:
def fitness_function(genotypes,genotype_count,df):
    accuracy_list = [] #Tworzę listę 
    for i in range(genotype_count): #dla każdego i tego genotypu
        df = pd.read_csv ('CollegeDistance.csv', index_col=0)
        df= usun_na(genotypes[i],df)
        new_df = usun_outliery(genotypes[i],df)
        new_df= prepare_data(new_df)
        
        #Tworzę zmienną y
        i = new_df.education > 14  
        new_df["DalszaEdukacja"]=np.where(i, 1,0) 
        X=new_df.drop(columns="education")
        y=X["DalszaEdukacja"]
        X=X.drop(columns="DalszaEdukacja")
        i=0
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=3)
        y_przewidywane = uzyskaj_wynik(genotypes[i], X_train, X_test, y_train, y_test)
        accuracy_list.append(print_accuracy(y_test,y_przewidywane))
    return accuracy_list

#### Turniej

In [28]:
#Turniej
def function_tournament(genotype_count,accuracy_list):
    parents=[]
    for i in range(2):
        para=random.sample(range(genotype_count), 2) # wybieramy pierwszych konkurentow - zwyciezca pojedynku zostaje rodzicem
        if accuracy_list[para[0]] > accuracy_list[para[1]]: 
            parents.append(para[0]) #jezeli suma jest większa dla konkurenta 0 to on zostaje rodzicem
        else:
            parents.append(para[1])    
    return parents #zwracam 


#### Krzyżowanie

In [29]:
#Funkcja Krzyżuj
def function_cross(genotypes,genotype_length,parents, child_population):
    cross_point=random.randrange(genotype_length-1) #Wylosuj punkt krzyżowania
    gensA = [] #lista genów I potomka
    gensB = [] #lista genow II potomka 
    #print(cross_point)
    if random.random()<0.9: #jezeli wylosowano ze ma byc krzyzowanie
        for i in range(0,cross_point): #od zerowego genu do miejsca krzyzowki
            gensA.append(genotypes[parents[0]][i]) 
            gensB.append(genotypes[parents[1]][i])  
        
        for i in range(cross_point,genotype_length-1): #od miejsca krzyzowki do ostatniego genu
            gensA.append(genotypes[parents[1]][i]) 
            gensB.append(genotypes[parents[0]][i]) 
        gensA.append(False) #Dodanie genów odpowiadających za mutację - domyślnie fałsz 
        gensB.append(False)
        child_population.append(gensA) #dodaj potomków do przekazanej listy z nową populacją
        child_population.append(gensB)
        #print(child_population[0]) 
    else: #gdy krzyzowanie nie zachodzi
        child_population.append(genotypes[parents[0]]) #jako potomkow dodaj po prostu rodzicow, bo brak zmian
        child_population.append(genotypes[parents[1]])    
    return child_population #zwroc uzupelniona liste

#### Mutacja

In [30]:
#Funkcja mutująca
def function_mutate(p_mute, genotype_count, child_population):
    for i in range(genotype_count): #Dla każdego z genotypów:
        pr = random.random() #Losuj prawdopodobieństwo mutowania
        if pr < p_mute: #Jeśli zachodzi mutacja..
            child_population[i][3]=True #..Ustawiam gen odpowiadający za mutowanie na true

    return child_population

### Pętla główna

#### Inicjacja

In [31]:

genotype_length = 4
genotype_count = 10
genotypes = []
p_mute = 0.6
liczba_pokolen = 3
for i in range(genotype_count):
    genotypes.append([random.randint(0, 2),random.randint(0, 2),random.randint(0, 3),False])    
for i in range(genotype_count):
        print(genotypes[i])
    
    

[0, 1, 2, False]
[0, 2, 3, False]
[2, 0, 0, False]
[0, 1, 3, False]
[0, 2, 2, False]
[2, 1, 2, False]
[2, 0, 3, False]
[2, 0, 0, False]
[2, 0, 2, False]
[0, 1, 0, False]


#### Przebieg właściwy

In [32]:
f = open('output.txt', 'w') 
df = pd.read_csv ('CollegeDistance.csv', index_col=0)
accuracy_list=fitness_function(genotypes,genotype_count,df)
poczatkowe_max = max(accuracy_list)
for licznik in range(liczba_pokolen):
    print("Pokolenie nr: ",licznik," ,liczba genotypów: ",genotype_count, file=f)
    for i in range(genotype_count): 
        print(genotypes[i], file=f) 
    print("Wartości funkcji dopasowania:", file=f)
    print(accuracy_list, file=f)    
    for i in range(4):
        index = accuracy_list.index(min(accuracy_list))
        accuracy_list.remove(min(accuracy_list))
        genotypes.remove(genotypes[index])
        genotype_count = genotypes.__len__()
        if genotype_count == 6:
            break
    print("Liczba genotypów po selekcji: ",genotype_count, file=f)    
    rodzice = []
    populacja_dzieci = []
    for i in range(5):
        rodzice.append(function_tournament(genotype_count,accuracy_list))
    for i in range(5):
                function_cross(genotypes,genotype_length,rodzice[i],populacja_dzieci)

    populacja_dzieci = function_mutate(p_mute,populacja_dzieci.__len__(),populacja_dzieci) #mutowanie
    accuracy_list_dzieci =fitness_function(populacja_dzieci,populacja_dzieci.__len__(),df) #otrzymaj listę sum
    print("Populacja dzieci pokolenia ",licznik, file=f)
    for i in range(populacja_dzieci.__len__()): 
        print(populacja_dzieci[i], file=f) 
    nowa_populacja = []
    accuracy_list_nowa_populacja = [] #w celu oszczędzenia liczby obliczeń odtwarzam również listę z wartościami dopasowania osobników
    for i in range(7): #WDopisujemy do nowej popualcji 7 dzieci o największej sumie
        index = accuracy_list_dzieci.index(max(accuracy_list_dzieci)) #znajdź indeks genotypu dziecka o najwększej sumie
        nowa_populacja.append(populacja_dzieci[index]) #dodaj dziecko o największej sumie do nowej populacji
        accuracy_list_nowa_populacja.append(accuracy_list_dzieci[index])
        accuracy_list_dzieci.remove(max(accuracy_list_dzieci)) #usuń go z listy sum
        populacja_dzieci.remove(populacja_dzieci[index]) #usuń go z listy dzieci
        


    index=accuracy_list.index(max(accuracy_list)) #znajdź indeks genotypu (rodzica) o najwększej sumie 
    nowa_populacja.append(genotypes[index])  #dodaj go do nowej populacji 
    accuracy_list_nowa_populacja.append(accuracy_list[index])
    genotype_count = nowa_populacja.__len__() 
    genotypes = nowa_populacja
    accuracy_list = accuracy_list_nowa_populacja
print("Wartość funkcji dopasowania otrzymana za pomocąprogramu genetycznego: ",max(accuracy_list),file=f)
print("Wartość funkcji dopasowania bez wykorzystania programu genetycznego: ",poczatkowe_max,file=f)        
f.close()    